In [3]:
import os
import pathlib
import re
import copy
import warnings
import time
import json
import pickle

from sys import getsizeof

import cv2
import xmltodict
import numpy as np
import pandas as pd
import tiffcapture as tc

from tifffile import imsave
from matplotlib import pyplot as pt

%matplotlib inline

In [4]:
POSITION = 40
PROCESS_PI = False
TIME_UNIT_FACTOR = 600
PIXEL_INCH_RATIO = .647
DIAMETER = 15
SYNCHRONIZED = True

# Function Defs

In [5]:
def removekey(d, key):
    """
    Remove a key from a  dictionary without destroying the reference
    to removed object (which might be used by other processes)
    """
    if key in d:
        r = dict(d)
        del r[key]
        return r

In [6]:
def parse_trees(links_tab, verbose=False, parse_velocity_displacement=False):
    """
    Given 'links in tab statistics' from TrackMate.
    Construct data structures representing trees ('tree') and
    their branching ('branch') respectively.
    
    -   'tree' contains mapping of TREE_ID (which is the same as
        the id of root's SPOT_ID) to its associated tree.
        Each tree is in turn a map of TREE_BRANCH_ID
        (which is the branch's first spot's SPOT_ID)
        to its branch which is represented as list of SPOT_ID's.
    -   'branch' encodes how the tree is structured. It contains
        mapping of TREE_ID to the branching configuration.
        The branching configuration is encoded as mapping of
        BRANCH_ID to its two children's BRANCH_ID.
    """
    
    links_tab = links_tab.copy()
    links_tab.index = links_tab.SPOT_TARGET_ID

    tree = {}
    branch = {}
    if parse_velocity_displacement:
        velocity = {}
        displacement = {}
        
    for track in links_tab.TRACK_ID.unique():
    
        sub = links_tab[links_tab.TRACK_ID == track]

        this_lines = {}
        this_branches = {}
        if parse_velocity_displacement:
            this_velocities = {}
            this_displacements = {}
        stack = []
    
        groot = sub.iloc[0][3]
        if verbose:
            print("Parsing tree %s" % groot)
        stack.append(groot)
    
        while len(stack) > 0:
        
            root = stack.pop()
            track = [root]
            if parse_velocity_displacement:
                # initial velocity and displacement is always 0
                vel = [0]
                disp = [0]
            
            if verbose:
                print("Parsing subtree %s" % root)
            nsub = sub[sub.SPOT_SOURCE_ID == root]
        
            while nsub.index.size > 0:
                if nsub.index.size == 1:
                    this = nsub.iloc[0][4]
                    track.append(this)
                    if parse_velocity_displacement:
                        ## TODO check whether the implementation is correct
                        if len(sub.loc[this]) == 13:
                            vel.append(sub.loc[this]['VELOCITY'])
                            disp.append(sub.loc[this]['DISPLACEMENT'])
                        else:
                            vel.append(sub.loc[this]['VELOCITY'].values[0])
                            disp.append(sub.loc[this]['DISPLACEMENT'].values[0])
                    nsub = sub[sub.SPOT_SOURCE_ID == this]
                else:
                    stack.append(nsub.iloc[0][4])
                    stack.append(nsub.iloc[1][4])
                    this_branches[root] = (nsub.iloc[0][4], nsub.iloc[1][4])
                    if verbose:
                        print("breaking")
                    break
                
            this_lines[root] = track
            if parse_velocity_displacement:
                this_velocities[root] = vel
                this_displacements[root] = disp
            if verbose:
                print("adding branch %s" % root)
    
        if verbose:
            print("Finishing...")
        tree[groot] = this_lines
        branch[groot] = this_branches
        if parse_velocity_displacement:
            velocity[groot] = this_velocities
            displacement[groot] = this_displacements
        
    if parse_velocity_displacement:
        return tree, branch, velocity, displacement
    else:
        return tree, branch

In [7]:
def extract_values(tree, spots_tab, colnames, verbose=False):
    """
    Extract varoius measurement values from spots statistics and save it
    in format similar to parsed tree. The list of values to be extracted
    from spots statistics is defined in 'colnames' 
    
    The information will be stored in following format:
    map(TYPE:(TREE_ID:TREE_BRANCH:list(VALUES))
    
    """
    
    vals = {}

    for tr in tree:
    
        if verbose:
            print("extracting values for %s" % tr)
    
        for colname in colnames:
        
            val_tree = {}
        
            for br in tree[tr]:
            
                brkeys = tree[tr][br]
                brvals = [sits[sits.ID == x][colname].values[0] for x in brkeys]
            
                val_tree[br] = brvals
            
            if colname not in vals:
                vals[colname] = {}
            
            vals[colname][tr] = val_tree
            
    return vals

In [8]:
def filter_trees_by_time(tree, branch, tree_values, min_time=0, max_time=10):
    """
    Filter out trees which measurement time start before or after
    the time as defined in 'min_time' and 'max_time'.
    
    Measurement time is defined as the return value from TrackMate
    encoded as 'POSITION_T'
    """
    
    tree_values_filtered = {}
    tree = copy.deepcopy(tree)
    branch = copy.deepcopy(branch)

    for v in tree_values.keys():
        tree_values_filtered[v] = {}

    for key in tree_values['POSITION_T'].keys():
    
        if (tree_values['POSITION_T'][key][key][0] >= min_time) and (tree_values['POSITION_T'][key][key][0] <= max_time):
            for k in tree_values_filtered.keys():
                tree_values_filtered[k][key] = tree_values[k][key]
        else:
            tree = removekey(tree, key)
            branch = removekey(branch, key)
                
    return  tree, branch, tree_values_filtered 

In [9]:
def get_video(path, to_rgb=False):
    """
    Read TIFF file containing multiple stacks (a video)
    and return sequential array of frame encoded as
    multidimensional array
    """
    
    tif = tc.opentiff(path)
    
    ## first image
    _, first_img = tif.retrieve()
    if to_rgb:
        first_img = cv2.cvtColor(first_img,cv2.COLOR_GRAY2RGB)

    ## slices
    pics = [first_img]

    for slide in tif:
        # convert grayscale to RGB
        if to_rgb:
            slide = cv2.cvtColor(slide, cv2.COLOR_GRAY2RGB)
        pics.append(slide)
    
    return pics

In [10]:
def extract_brightness(tree_id, 
                       tree, 
                       tree_values, 
                       reference_vid_path, 
                       diameter=DIAMETER, 
                       caspase=True, 
                       normalize=True, 
                       synchronized=False):
    """
    Given cell death signal video, extract brightness level for each position
    of tracked cell trees, if such poisition coincides with the.
    
    This function returns:
    -   the brightness level of each spot in the tree, if such is measured.
        The encoding follows that of the tree itself
    -   cell death signal time-lapsed video of containing the cell belonging
        to the 'tree_id'. Useful for debugging/sanity testing 
    """
    
    pics = get_video(reference_vid_path)
    pics_intersect = []

    t_time = tree_values['POSITION_T'][tree_id] # tree_values
    t_x = tree_values['POSITION_X'][tree_id]
    t_y = tree_values['POSITION_Y'][tree_id]
    t_tree = tree[tree_id]

    t_brightness = {}
    
    if normalize:
        base_brightness = []
        for pic in pics:
            base_brightness.append(cv2.sumElems(pic)[0] / float(pic.shape[0] * pic.shape[1]))

    for br in t_tree:
        brch = t_tree[br]
        x = t_x[br]
        y = t_y[br]
        time = t_time[br]
        brightness = []
    
        for i in range(len(brch)):
            if synchronized or (not caspase):
                if time[i] % 1800 == 0:
                    pos = time[i] // 1800
                    pic_base = pics[pos]
                    pic_bg = np.zeros(pic_base.shape, np.uint16)
                    pic_bg = cv2.circle(pic_bg, 
                                       (int(x[i] / PIXEL_INCH_RATIO), int(y[i] / PIXEL_INCH_RATIO)),
                                       int(diameter / PIXEL_INCH_RATIO),
                                       (255, 255, 255),
                                       -1)
                    pic = cv2.bitwise_and(pic_base, pic_bg)
                    if normalize:
                        brightness.append(cv2.sumElems(pic)[0] / (base_brightness[pos] + .0001))
                    else:
                        brightness.append(cv2.sumElems(pic)[0])
                    pics_intersect.append(pic)
                else:
                    brightness.append(np.nan)     
            else:
                if ((time[i] - 76200) >= 0) & ((time[i] - 76200) % 1800 == 0):
                    pos = (time[i] - 76200) // 1800
                    pic_base = pics[pos]
                    pic_bg = np.zeros(pic_base.shape, np.uint16)
                    pic_bg = cv2.circle(pic_bg, 
                                       (int(x[i] / PIXEL_INCH_RATIO), int(y[i] / PIXEL_INCH_RATIO)),
                                       int(diameter / PIXEL_INCH_RATIO),
                                       (255, 255, 255),
                                       -1)
                    pic = cv2.bitwise_and(pic_base, pic_bg)
                    if normalize:
                        brightness.append(cv2.sumElems(pic)[0] / (base_brightness[pos] + .0001))
                    else:
                        brightness.append(cv2.sumElems(pic)[0])
                    pics_intersect.append(pic)
                else:
                    brightness.append(np.nan)
            
        t_brightness[br] = brightness
    return t_brightness, np.array(pics_intersect)

In [11]:
def extract_brightness_template(tree_id, 
                                tree, 
                                tree_values, 
                                pics, 
                                diameter=DIAMETER, 
                                caspase=True, 
                                normalize=True,
                                synchronized=False):
    """
    Similar to extract_brightness(), except that it
    takes video template form the user
    
    This function returns:
    -   the brightness level of each spot in the tree, if such is measured.
        The encoding follows that of the tree itself
    -   cell death signal time-lapsed video of containing the cell belonging
        to the 'tree_id'. Useful for debugging/sanity testing 
    """
    
    pics_intersect = []

    t_time = tree_values['POSITION_T'][tree_id] # tree_values
    t_x = tree_values['POSITION_X'][tree_id]
    t_y = tree_values['POSITION_Y'][tree_id]
    t_tree = tree[tree_id]

    t_brightness = {}
    
    if normalize:
        base_brightness = []
        for pic in pics:
            base_brightness.append(cv2.sumElems(pic)[0] / float(pic.shape[0] * pic.shape[1]))

    for br in t_tree:
        brch = t_tree[br]
        x = t_x[br]
        y = t_y[br]
        time = t_time[br]
        brightness = []
    
        for i in range(len(brch)):
            if synchronized or (not caspase):
                if time[i] % 1800 == 0:
                    pos = time[i] // 1800
                    pic_base = pics[pos]
                    pic_bg = np.zeros(pic_base.shape, np.uint16)
                    pic_bg = cv2.circle(pic_bg, 
                                       (int(x[i] / PIXEL_INCH_RATIO), int(y[i] / PIXEL_INCH_RATIO)),
                                       int(diameter / PIXEL_INCH_RATIO),
                                       (255, 255, 255),
                                       -1)
                    pic = cv2.bitwise_and(pic_base, pic_bg)
                    if normalize:
                        brightness.append(cv2.sumElems(pic)[0] / (base_brightness[pos] + .0001))
                    else:
                        brightness.append(cv2.sumElems(pic)[0])
                    pics_intersect.append(pic)
                else:
                    brightness.append(np.nan)
            else:
                if ((time[i] - 76200) >= 0) & ((time[i] - 76200) % 1800 == 0):
                    pos = (time[i] - 76200) // 1800
                    pic_base = pics[pos]
                    pic_bg = np.zeros(pic_base.shape, np.uint16)
                    pic_bg = cv2.circle(pic_bg, 
                                       (int(x[i] / PIXEL_INCH_RATIO), int(y[i] / PIXEL_INCH_RATIO)),
                                       int(diameter / PIXEL_INCH_RATIO),
                                       (255, 255, 255),
                                       -1)
                    pic = cv2.bitwise_and(pic_base, pic_bg)
                    if normalize:
                        brightness.append(cv2.sumElems(pic)[0] / (base_brightness[pos] + .0001))
                    else:
                        brightness.append(cv2.sumElems(pic)[0])
                    pics_intersect.append(pic)
                else:
                    brightness.append(np.nan)
            
        t_brightness[br] = brightness
    return t_brightness, np.array(pics_intersect)

In [12]:
def assign_tree_to_contours(tree_values, contours):
    """
    Given tree values and recognized contours, compute:
    
    -   'assocs': table listing recognized slit and the TREE_ID
        of singly-placed cell tree located in the slit
    -   'occupancy': list containing the number of cell trees
        located in a slit. NOTE: the list is not associated with
        ordering encoded in 'assocs'
    """

    occuppancy = {x:0 for x in range(len(contours))}
    cell_trees = []
    slits = []

    for tr in tree_values['POSITION_X'].keys():
    
        cell_trees.append(tr)

        x = tree_values['POSITION_X'][tr][tr][0] / PIXEL_INCH_RATIO
        y = tree_values['POSITION_Y'][tr][tr][0] / PIXEL_INCH_RATIO
    
        counter = 0
        match = 0
        matchloc = None
    
        for ct in cts:
        
            if cv2.pointPolygonTest(ct, (x, y), False) > 0:
                occuppancy[counter] += 1
                matchloc = counter
                match += 1
            counter +=  1
        
        if match == 0:
            slits.append(None)
        elif match == 1:
            slits.append(matchloc)
        else:
            print("Tree %s got too many matches" % tr)
        
    assocs = pd.DataFrame({'CELL_LINE': cell_trees, 'SLIT_ID': slits})
    occuppancy = np.array(list(occuppancy.values()))
    
    return assocs, occuppancy

In [13]:
def get_contour(contour_mask_path, min_size=7000, max_size=10000):
    """
    Give contour mask picture (created by adjusting contrast/brightness
    followed by RATS and "fill holes" command) detect contours of
    slits annd return:
    - the list of filtered contours
    - the size list of filtered contours
    - the size of all detected contours (for testing purpose)
    """
    
    ## collect contours and wrap into a function

    im = cv2.imread(contour_mask_path)
    gray= cv2.cvtColor(im,cv2.COLOR_BGR2GRAY)
    im2, contours, hierarchy  = cv2.findContours(gray,cv2.RETR_LIST,cv2.CHAIN_APPROX_SIMPLE)

    all_cts_area = np.array([cv2.contourArea(x) for x  in contours])

    filtered_cts = []
    filtered_cts_area = []

    for ct in contours:
        ct_area = cv2.contourArea(ct)
        if (ct_area <= max_size) and (ct_area >= min_size):
            filtered_cts.append(ct)
            filtered_cts_area.append(ct_area)
            
    return filtered_cts, filtered_cts_area, all_cts_area


# Pipeline

In [14]:
"""
TREE_ID:
    "TREE"
    "BRANCH"
    "POSITION_T"
    "POSITION_X"
    "POSITION_Y"
    "CASPASE"
    "CASPASE_NORM"
    "CASPASE_SUB"
    "CASPASE_SUB_NORM"
    "PI"
    "PI_NORM"
    "PI_SUB"
    "PI_SUB_NORM"
"""

'\nTREE_ID:\n    "TREE"\n    "BRANCH"\n    "POSITION_T"\n    "POSITION_X"\n    "POSITION_Y"\n    "CASPASE"\n    "CASPASE_NORM"\n    "CASPASE_SUB"\n    "CASPASE_SUB_NORM"\n    "PI"\n    "PI_NORM"\n    "PI_SUB"\n    "PI_SUB_NORM"\n'

# Export Filtered Data

In [15]:
done = []
dat = {}
dat_f = {}

In [16]:
positions = list(set(list(range(64)[1:])) - set([12, 26, 37, 38, 44, 60]))

In [17]:
## TODO: continue

diameter = 15

for position in positions:
    
    if position < 10:
        pos = "0%d" % position
    else:
        pos = str(position)
    
    if position not in done:


        t = time.time()
        print("Processing position %s...\n  Reading files" % pos)

#         track_path = "../test/eli-new-sync-bf-%s/out-focus/out/tracked/" % pos
#         caspase_path = "../test/eli-new-sync-bf-%s/caspase/seq0001xy%sc1.tif" % (pos, pos)
#         caspase_sub_path = "../test/eli-new-sync-bf-%s/caspase/seq0001xy%sc1_sub.tif" % (pos, pos)
#         pi_path = "../test/eli-new-sync-bf-%s/pi/seq0002xy%sc1.tif" % (pos, pos)
#         pi_sub_path = "../test/eli-new-sync-bf-%s/pi/sub.tif" % pos
        
        track_path = "E:/sync_processed/eli-new-sync-bf-%s/out-focus/out/tracked/" % pos
        caspase_path = "E:/sync_processed/eli-new-sync-bf-%s/caspase/seq0001xy%sc1.tif" % (pos, pos)
        caspase_sub_path = "E:/sync_processed/eli-new-sync-bf-%s/caspase/seq0001xy%sc1_sub.tif" % (pos, pos)
        pi_path = "E:/sync_processed/eli-new-sync-bf-%s/pi/seq0002xy%sc1.tif" % (pos, pos)
        pi_sub_path = "E:/sync_processed/eli-new-sync-bf-%s/pi/sub.tif" % pos

        ba = pd.read_csv(os.path.join(track_path, 'Links in tracks statistics.csv'))
        sits = pd.read_csv(os.path.join(track_path, 'Spots in tracks statistics.csv'))
        print("  Reading files done. Time elapsed: %f" % (time.time() - t))

        ## parsing tree
        t = time.time()
        print("  Parsing trees and values")
        tree, branch, velocity, displacement = parse_trees(ba, parse_velocity_displacement=True)
        svals = extract_values(tree, sits, ['POSITION_T', 'POSITION_X', 'POSITION_Y', 'TOTAL_INTENSITY', 'QUALITY'])
        print("  Parsing done. Time elapsed: %f" % (time.time() - t))

        ## filter tree

        t = time.time()
        print("  Filtering tree")
        tree, branch, svals = filter_trees_by_time(tree, branch, svals)
        print("  Filtering done. Time elapsed: %f" % (time.time() - t))
        
        ## extract cell association
        t = time.time()
        print("  Getting contour")
        
#         path = "../test/sync_processed/eli-new-sync-bf-%s/in-focus/" % pos
        path = "E:/sync_processed/eli-new-sync-bf-%s/in-focus/" % pos
        impath = "seq0000xy%sc1-mask.tif" % pos
        
        cts, _, _ = get_contour(os.path.join(path, impath))
        assocs, _ = assign_tree_to_contours(svals, cts)
        cell_trees = set(assocs.CELL_LINE[~assocs.SLIT_ID.isnull()].tolist())

        print("  Contour recognition done. Time elapsed: %f" % (time.time() - t))
        ## extract cas + pi value

        print("  Extarcting caspase and pi measurement values")
        t = time.time()
        caspases = {}
        caspases_norm = {}
        caspases_sub = {}
        caspases_sub_norm = {}
        pis = {}
        pis_norm = {}
        pis_sub = {}
        pis_sub_norm = {}
    
        pics_casp = get_video(caspase_path)
        pics_casp_sub = get_video(caspase_sub_path)
        pics_pi = get_video(pi_path)
        pics_pi_sub = get_video(pi_sub_path)
    
        for k in tree.keys():

            cs, _ = extract_brightness_template(k, 
                                                tree, 
                                                svals, 
                                                pics_casp, 
                                                caspase=True,
                                                normalize=False,
                                                synchronized=SYNCHRONIZED)
            caspases[k] = cs
    
            csnorm, _ = extract_brightness_template(k, 
                                                    tree, 
                                                    svals, 
                                                    pics_casp, 
                                                    caspase=True, 
                                                    normalize=True, 
                                                    synchronized=SYNCHRONIZED)
            caspases_norm[k] = csnorm
    
            cssub, _ = extract_brightness_template(k, 
                                                   tree, 
                                                   svals, 
                                                   pics_casp_sub, 
                                                   caspase=True,
                                                   normalize=False,
                                                   synchronized=SYNCHRONIZED)
            caspases_sub[k] = cssub
    
            cssubnorm, _ = extract_brightness_template(k, 
                                                       tree, 
                                                       svals, 
                                                       pics_casp_sub, 
                                                       caspase=True, 
                                                       normalize=True, 
                                                       synchronized=SYNCHRONIZED)
            caspases_sub_norm[k] = cssubnorm
                
    
        print("  Extracting caspase done. Time elapsed: %f" % (time.time() - t))
    
        t = time.time()
        for k in tree.keys():
    
            pi, _ = extract_brightness_template(k, 
                                                tree, 
                                                svals, 
                                                pics_pi, 
                                                caspase=False, 
                                                normalize=False)
            pis[k] = pi
        
            pinorm, _ = extract_brightness_template(k, 
                                                    tree, 
                                                    svals, 
                                                    pics_pi, 
                                                    caspase=False, 
                                                    normalize=True)
            pis_norm[k] = pinorm
        
            pisub, _ = extract_brightness_template(k, 
                                                   tree, 
                                                   svals, 
                                                   pics_pi_sub, 
                                                   caspase=False, 
                                                   normalize=False)
            pis_sub[k] = pisub
        
            pisubnorm, _ = extract_brightness_template(k, 
                                                       tree, 
                                                       svals, 
                                                       pics_pi_sub, 
                                                       caspase=False, 
                                                       normalize=True)
            pis_sub_norm[k] = pisubnorm
    
        print("  Extracting pi done. Time elapsed: %f" % (time.time() - t))

        print("  Wrapping values")
        t = time.time()
        
        pos_dat = {}
        pos_dat_f = {}
        pos_dat_trees = {}
        pos_dat_trees_f = {}        
        for tr in tree.keys():
            t_tree = {
                "TREE": tree[tr],
                "BRANCH": branch[tr],
                "POSITION_T": svals["POSITION_T"][tr],
                "POSITION_X": svals["POSITION_X"][tr],
                "POSITION_Y": svals["POSITION_Y"][tr],
                "CASPASE": caspases[tr],
                "CASPASE_NORM": caspases_norm[tr],
                "CASPASE_SUB": caspases_sub[tr],
                "CASPASE_SUB_NORM": caspases_sub_norm[tr],
                "PI": pis[tr],
                "PI_NORM": pis_norm[tr],
                "PI_SUB": pis_sub[tr],
                "PI_SUB_NORM": pis_sub_norm[tr],
                "VELOCITY": velocity[tr],
                "DISPLACEMENT": displacement[tr]
            }
            if tr in cell_trees:
                pos_dat_trees_f[tr] = t_tree
                pos_dat_trees[tr] = t_tree
            else:
                pos_dat_trees[tr] = t_tree
    
        pos_dat["TREES"] = pos_dat_trees
        pos_dat["GAP_PER_FRAME_SECONDS"] = 600
        pos_dat["PIXEL_TO_INCH"] = 1 / .647
        dat[pos] = pos_dat
        
        pos_dat_f["TREES"] = pos_dat_trees_f
        pos_dat_f["GAP_PER_FRAME_SECONDS"] = 600
        pos_dat_f["PIXEL_TO_INCH"] = 1 / .647
        dat_f[pos] = pos_dat_f
        
        done.append(position)
        
        for tree_id in cell_trees:    
            dat_f[pos]["TREES"][tree_id]["SLIT_ID"] = int(assocs[assocs.CELL_LINE == tree_id]["SLIT_ID"].values[0])

        print("  Extraction done. Time elapsed: %f" % (time.time() - t))
    else:
        print("  Position %s already extracted" % pos)

Processing position 01...
  Reading files


OSError: File b'E:/sync_processed/eli-new-sync-bf-01/out-focus/out/tracked/Links in tracks statistics.csv' does not exist

## Making filtered data JSON compliant and export it

In [ ]:
dat_c = copy.deepcopy(dat)
dat_fc = copy.deepcopy(dat_f)

In [ ]:
print("Making dat JSON compliant")
for pos in dat_c.keys():
    t = time.time()
    print("  Doing position %s" % pos)

    for tid in dat_c[pos]['TREES'].keys():
    ## change value of single tree
        dat_c[pos]['TREES'][tid]['BRANCH'] = {str(k):(int(v[0]), int(v[1])) for k,v in dat_c[pos]['TREES'][tid]['BRANCH'].items()}
        dat_c[pos]['TREES'][tid]['TREE'] = {str(k):[int(val) for val in v] for k,v in dat_c[pos]['TREES'][tid]['TREE'].items()}
        dat_c[pos]['TREES'][tid]['POSITION_T'] = {str(k):[int(val) for val in v] for k,v in dat_c[pos]['TREES'][tid]['POSITION_T'].items()}
        dat_c[pos]['TREES'][tid]['POSITION_X'] = {str(k):v for k,v in dat_c[pos]['TREES'][tid]['POSITION_X'].items()}
        dat_c[pos]['TREES'][tid]['POSITION_Y'] = {str(k):v for k,v in dat_c[pos]['TREES'][tid]['POSITION_Y'].items()}
        dat_c[pos]['TREES'][tid]['CASPASE'] = {str(k):v for k,v in dat_c[pos]['TREES'][tid]['CASPASE'].items()}
        dat_c[pos]['TREES'][tid]['CASPASE_NORM'] = {str(k):v for k,v in dat_c[pos]['TREES'][tid]['CASPASE_NORM'].items()}
        dat_c[pos]['TREES'][tid]['CASPASE_SUB'] = {str(k):v for k,v in dat_c[pos]['TREES'][tid]['CASPASE_SUB'].items()}
        dat_c[pos]['TREES'][tid]['CASPASE_SUB_NORM'] = {str(k):v for k,v in dat_c[pos]['TREES'][tid]['CASPASE_SUB_NORM'].items()}
        dat_c[pos]['TREES'][tid]['PI'] = {str(k):v for k,v in dat_c[pos]['TREES'][tid]['PI'].items()}
        dat_c[pos]['TREES'][tid]['PI_NORM'] = {str(k):v for k,v in dat_c[pos]['TREES'][tid]['PI_NORM'].items()}
        dat_c[pos]['TREES'][tid]['PI_SUB'] = {str(k):v for k,v in dat_c[pos]['TREES'][tid]['PI_SUB'].items()}
        dat_c[pos]['TREES'][tid]['PI_SUB_NORM'] = {str(k):v for k,v in dat_c[pos]['TREES'][tid]['PI_SUB_NORM'].items()}
        dat_c[pos]['TREES'][tid]['VELOCITY'] = {str(k):v for k,v in dat_c[pos]['TREES'][tid]['VELOCITY'].items()}
        dat_c[pos]['TREES'][tid]['DISPLACEMENT'] = {str(k):v for k,v in dat_c[pos]['TREES'][tid]['DISPLACEMENT'].items()}

    ## change value of trees
    dat_c[pos]['TREES'] = {str(k):v for k,v in dat_c[pos]['TREES'].items()}
    
    print("  Finished. Time: %f" % (time.time() - t))

In [ ]:
print("Making dat JSON compliant")
for pos in dat_fc.keys():
    t = time.time()
    print("  Doing position %s" % pos)

    for tid in dat_fc[pos]['TREES'].keys():
    ## change value of single tree
        dat_fc[pos]['TREES'][tid]['BRANCH'] = {str(k):(int(v[0]), int(v[1])) for k,v in dat_fc[pos]['TREES'][tid]['BRANCH'].items()}
        dat_fc[pos]['TREES'][tid]['TREE'] = {str(k):[int(val) for val in v] for k,v in dat_fc[pos]['TREES'][tid]['TREE'].items()}
        dat_fc[pos]['TREES'][tid]['POSITION_T'] = {str(k):[int(val) for val in v] for k,v in dat_fc[pos]['TREES'][tid]['POSITION_T'].items()}
        dat_fc[pos]['TREES'][tid]['POSITION_X'] = {str(k):v for k,v in dat_fc[pos]['TREES'][tid]['POSITION_X'].items()}
        dat_fc[pos]['TREES'][tid]['POSITION_Y'] = {str(k):v for k,v in dat_fc[pos]['TREES'][tid]['POSITION_Y'].items()}
        dat_fc[pos]['TREES'][tid]['CASPASE'] = {str(k):v for k,v in dat_fc[pos]['TREES'][tid]['CASPASE'].items()}
        dat_fc[pos]['TREES'][tid]['CASPASE_NORM'] = {str(k):v for k,v in dat_fc[pos]['TREES'][tid]['CASPASE_NORM'].items()}
        dat_fc[pos]['TREES'][tid]['CASPASE_SUB'] = {str(k):v for k,v in dat_fc[pos]['TREES'][tid]['CASPASE_SUB'].items()}
        dat_fc[pos]['TREES'][tid]['CASPASE_SUB_NORM'] = {str(k):v for k,v in dat_fc[pos]['TREES'][tid]['CASPASE_SUB_NORM'].items()}
        dat_fc[pos]['TREES'][tid]['PI'] = {str(k):v for k,v in dat_fc[pos]['TREES'][tid]['PI'].items()}
        dat_fc[pos]['TREES'][tid]['PI_NORM'] = {str(k):v for k,v in dat_fc[pos]['TREES'][tid]['PI_NORM'].items()}
        dat_fc[pos]['TREES'][tid]['PI_SUB'] = {str(k):v for k,v in dat_fc[pos]['TREES'][tid]['PI_SUB'].items()}
        dat_fc[pos]['TREES'][tid]['PI_SUB_NORM'] = {str(k):v for k,v in dat_fc[pos]['TREES'][tid]['PI_SUB_NORM'].items()}
        dat_fc[pos]['TREES'][tid]['VELOCITY'] = {str(k):v for k,v in dat_fc[pos]['TREES'][tid]['VELOCITY'].items()}
        dat_fc[pos]['TREES'][tid]['DISPLACEMENT'] = {str(k):v for k,v in dat_fc[pos]['TREES'][tid]['DISPLACEMENT'].items()}

    ## change value of trees
    dat_fc[pos]['TREES'] = {str(k):v for k,v in dat_fc[pos]['TREES'].items()}
    
    print("  Finished. Time: %f" % (time.time() - t))

In [ ]:
with open('../results/syn-filtered.json', 'w') as fp:
    json.dump(dat_fc, fp)

In [ ]:
pickle.dump(dat_fc, open("../results/syn-filtered.p", "wb" ) )

In [ ]:
with open('../results/syn-unfiltered.json', 'w') as fp:
    json.dump(dat_c, fp)

In [ ]:
pickle.dump(dat_c, open("../results/syn-unfiltered.p", "wb" ) )